In [2]:
from __future__  import annotations
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, cycle, product, islice, chain
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional
from sys         import maxsize

import re
import ast
import operator

import numpy as np

In [3]:
def read_data(input: str, parser=str, sep='\n', testing=False) -> list:
    if testing:
        sections = input.split(sep)
    else:
        sections = open(input).read().split(sep)
    return [parser(section) for section in sections]

In [4]:
test_string = "389125467"
test_ins = read_data(test_string, parser=str, sep=None, testing=True)

In [61]:
def run_part1(ins: List[str], num_iters=100) -> int:
    cups = list(int(x) for x in chain(*ins))
    curr_set = set(sorted(cups, reverse=True))
    # print("cups in order are ", cups)
    # print("cups set ", curr_set)

    for _ in range(num_iters):
        curr_cup = cups.pop(0)
        picked_cups = [cups.pop(0) for _ in range(3)]
        iter_cups = sorted(curr_set - set(picked_cups), reverse=True)

        # print("curr cup", curr_cup)
        # print("picked cups", picked_cups)
        # print("iter cups", iter_cups)

        # print("remaining", cups)
        
        destination = get_next(iter_cups, curr_cup)
        # print("destination", destination)
        insert_pt = cups.index(destination) + 1
        # print("insert pt", insert_pt)

        cups = cups[:insert_pt] + picked_cups + cups[insert_pt:] + [curr_cup]
        # print("new order", cups)
    
    one_idx = cups.index(1)
    cups_order = cups[one_idx+1:] + cups[:one_idx]
    return int("".join(map(str, cups_order)))

def get_next(ins: List[int], starts_with: int) -> int:
    '''returns destination cup'''
    ins = deque(ins)
    # move to the starts with number
    while ins[0] != starts_with:
        ins.rotate(-1)
    # move current number to the back
    ins.rotate(-1)
    return ins[0]

In [63]:
run_part1(test_ins, num_iters=100)

67384529

Part I  

Using your labeling, simulate 100 moves. What are the labels on the cups after cup 1?

In [64]:
real_ins = read_data("input.txt", parser=str, sep=None)
run_part1(real_ins)

72496583

Part II

Determine which two cups will end up immediately clockwise of cup 1. What do you get if you multiply their labels together?

In [194]:
class Deque(deque):
    def split_at(self, idx: int):
        left = Deque()
        for _ in range(idx):
            left.append(self.popleft())
        return left, self

def run_part2(ins: List[str], num_iters=100, total_nums=10) -> int:
    cups = Deque(int(x) for x in chain(*ins))
    print(cups)
    if total_nums > 10:
        extra = Deque(range(11, total_nums + 1))
        extra.extendleft(reversed(cups))
        cups = extra
        print(list(islice(cups, 0, total_nums)))
    curr_set = Deque(sorted(cups, reverse=True))
    # print("cups in order are ", cups)
    # print("cups set ", curr_set)

    for i in range(num_iters):
        if i % 100000 == 0:
            print(i, "out of", num_iters)
        # need pop with O(1)
        curr_cup = cups.popleft()
        picked_cups = Deque([cups.popleft() for _ in range(3)])
        # iter_cups = sorted(curr_set - set(picked_cups), reverse=True)

        # print("curr cup", curr_cup)
        # print("picked cups", picked_cups)
        # print("iter cups", iter_cups)

        # print("remaining", cups)
        
        destination = get_next2(curr_set, picked_cups, curr_cup)
        # print("destination", destination)
        insert_pt = cups.index(destination) + 1
        # print("insert pt", insert_pt)

        # O(1) insertion method
        left, right = cups.split_at(insert_pt)
        left.extend(picked_cups)
        right.append(curr_cup)
        right.extendleft(reversed(left))
        # right.appendleft(dest_cup)
        cups  = right
        # cups = cups, 0, insert_pt) + picked_cups + islice(cups, insert_pt, -1) + [curr_cup]
        # print("new order", cups)
    
    one_idx = cups.index(1)
    cups_order = list(islice(cups, one_idx+1, None)) + list(islice(cups, 0, one_idx))
    return int("".join(map(str, cups_order)))

def get_next2(ins: List[int], excluded: List[int], starts_with: int) -> int:
    '''returns destination cup
    takes in a deques as ins'''
    # print("in getting next")
    # print(ins, excluded, starts_with)

    # move to the starts with number
    while ins[0] != starts_with:
        # print(ins)
        ins.rotate(-1)
    # move current number to the back
    ins.rotate(-1)
    while ins[0] in excluded:
        ins.rotate(-1)
    return ins[0]

In [195]:
# %%timeit
# run_part2(test_ins, num_iters=int(10e6))

In [ ]:
run_part2(test_ins, num_iters=int(10e6), total_nums=1000)

crap still not fast enough: 21.3 s with crazy method

Took 22 seconds with the regular method

In [197]:
run_part2(test_ins, num_iters=int(10e6), total_nums=1000)

Deque([3, 8, 9, 1, 2, 5, 4, 6, 7])
[3, 8, 9, 1, 2, 5, 4, 6, 7, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 2

KeyboardInterrupt: 

In [160]:
run_part2(test_ins, num_iters=int(100))

67384529

In [100]:
# run_part2(real_ins)